In [1]:
import pandas as pd
import time
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession, types
import pyspark.sql.functions as F
from  pyspark.sql.functions import col
data_filepath = "../../data/steam_reviews.csv"

In [2]:
steam_reviews_schema = types.StructType([
    types.StructField('_c0', types.StringType()),
    types.StructField('app_id', types.StringType()),
    types.StructField('app_name', types.StringType()),
    types.StructField('review_id', types.StringType()),
    types.StructField('language', types.StringType()),
    types.StructField('review', types.StringType()),
    types.StructField('timestamp_created', types.StringType()),
    types.StructField('timestamp_updated', types.StringType()),
    types.StructField('recommended', types.BooleanType()),
    types.StructField('votes_helpful', types.IntegerType()),
    types.StructField('votes_funny', types.IntegerType()),
    types.StructField('weighted_vote_score', types.FloatType()),
    types.StructField('comment_count', types.IntegerType()),
    types.StructField('steam_purchase', types.BooleanType()),
    types.StructField('received_for_free', types.BooleanType()),
    types.StructField('written_during_early_access', types.BooleanType()),
    types.StructField('author.steamid', types.StringType()),
    types.StructField('author.num_games_owned', types.IntegerType()),
    types.StructField('author.num_reviews', types.IntegerType()),
    types.StructField('author.playtime_forever', types.FloatType()),
    types.StructField('author.playtime_last_two_weeks', types.FloatType()),
    types.StructField('author.playtime_at_review', types.FloatType()),
    types.StructField('author.last_played', types.FloatType()),
])


In [3]:
#spark = SparkSession.builder.appName('games').getOrCreate()
spark = (
    SparkSession.builder.config("spark.sql.debug.maxToStringFields", 100)
    .appName("reviews")
    .getOrCreate()
)

steam_reviews = spark.read.format('csv') \
                    .schema(steam_reviews_schema) \
                    .option('header', 'true') \
                    .load(data_filepath)

# steam_reviews.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/02 15:46:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/02 15:46:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
originalSize = steam_reviews.count()

steam_reviews = steam_reviews.where(col("app_id").isNotNull() \
                                    & col("app_name").isNotNull() \
                                    & (col("app_id") != "0") \
                                    & (col("app_name") != "0") \
                                    & (col("app_name").cast("int").isNull()) \
                                    & (col("app_id").cast("int").isNotNull()) \
                                    & (col("app_id").cast("int") != 0)  )

steam_reviews = steam_reviews.withColumnRenamed("author.steamid", "author_steamid") \
                             .withColumnRenamed("author.num_games_owned", "author_num_games_owned") \
                             .withColumnRenamed("author.num_reviews", "author_num_reviews") \
                             .withColumnRenamed("author.playtime_forever", "author_playtime_forever") \
                             .withColumnRenamed("author.playtime_last_two_weeks", "author_playtime_last_two_weeks") \
                             .withColumnRenamed("author.playtime_at_review", "author_playtime_at_review") \
                             .withColumnRenamed("author.last_played", "author_last_played")

steam_reviews = steam_reviews.drop("_c0")

all_games = steam_reviews.select(col("app_id"), col("review_id")) \
                         .groupBy(col("app_id")).agg(F.count("review_id").alias("review_count")) \
                         .where(col("review_count") >= 500) \
                         .select("app_id", "review_count")

# steam_reviews = steam_reviews.join(all_games.hint("broadcast"), ["app_id"])

# currentSize = steam_reviews.count()
# gameCount = all_games.count()

# print("Number of reviews:")
# print("  Original data size:", originalSize)
# print("  Current data size:", currentSize)
# print("  Data Removed:", originalSize - currentSize)
# print("")
# print("Number of games:", gameCount)
# print("")

# steam_reviews.show(100)

# steam_reviews.write.csv("output", mode='overwrite')



In [5]:
steam_reviews = steam_reviews.select("app_id", "app_name").dropDuplicates(['app_id'])

In [13]:
review_counts = steam_reviews.join(all_games.hint("broadcast"), ["app_id"]) \
                                 .select("app_id", "app_name", "review_count") \
                                 .orderBy(col("app_id").cast(types.IntegerType()))

review_counts.show(10)

+------+--------------------+------------+
|app_id|            app_name|review_count|
+------+--------------------+------------+
|    70|           Half-Life|       57320|
|   240|Counter-Strike: S...|      118081|
|   420|Half-Life 2: Epis...|       23501|
|   620|            Portal 2|      232329|
|  2870|           X Rebirth|        7001|
|  4000|         Garry's Mod|      655524|
|  7510|            X-Blades|        2805|
|  8870|   BioShock Infinite|      107277|
|  8930|Sid Meier's Civil...|      171404|
| 32470|STAR WARS™ Empire...|       20553|
+------+--------------------+------------+
only showing top 10 rows



In [8]:
len(review_counts.collect())

314

In [ ]:
all_games = steam_reviews.select(col("app_id"), col("review_id")) \
                         .groupBy(col("app_id")).agg(F.count("review_id").alias("review_count")) \
                         .where(col("review_count") >= 500) \
                         .select("app_id")
                        #  .orderBy(col("review_count"))

In [ ]:
steam_reviews = steam_reviews.join(all_games.hint("broadcast"), ["app_id"])

In [ ]:
test.show(100)

In [ ]:
afterSize = test.count()


print("Original data size:", originalSize)
print("Current data size:", afterSize)
print("Total Removed:", originalSize - afterSize)
print("Little Game Removed:", currentSize - afterSize)

In [ ]:
test = steam_reviews.where(col("app_id") == "619290")
test.show(100)

In [ ]:
steam_reviews.show(100)

# Ignore

In [ ]:
unwanted = steam_reviews.select(col("app_id")).where(col("app_id").isNull() \
                              | (F.lower(col("app_id")) == "null") \
                              | (col("app_id") == "") \
                              | (col("app_id") == "0"))

In [ ]:
unwanted.show(10)

In [ ]:
unwanted = steam_reviews.select(col("app_name")).where(col("app_name").isNull() \
                              | (F.lower(col("app_name")) == "null") \
                              | (col("app_name") == "") \
                              | (col("app_name") == "0"))

In [ ]:
unwanted.show(10)